In [6]:
!pip install duckdb
!pip install notebook
!pip install jupysql pandas matplotlib duckdb-engine


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import os
osm_dataset = "cemetery"
input_path = f"./data/geojson/{osm_dataset}.geojson"
input_filename = os.path.basename(input_path)

In [10]:
import duckdb
from sqlalchemy.engine import create_engine

%load_ext sql
conn = create_engine('duckdb:///data/local.duckdb')
%sql conn --alias duckdb
%config SqlMagic.named_parameters="enabled" 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;


Running query in 'duckdb'

Success


In [12]:
%%sql
CREATE OR REPLACE TABLE point_data AS
SELECT *
FROM ST_Read("{{input_path}}")

Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [13]:

import random
import string

try:
    conn.raw_connection().remove_function('random_string')
except:
    pass

# Define the Python function to generate a random string of length 'n'
def random_string(n: int) -> str:
    return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# Register the function with DuckDB

conn.raw_connection().create_function('random_string', random_string, side_effects=True)

In [14]:
%sql SELECT random_string(10) AS random_string;

Running query in 'duckdb'

random_string
sIVqnfUp64


In [17]:

%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT
        rowid,
        centroid: geom.ST_Centroid(),
    ORDER BY hash(rowid + 43) DESC
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    location: 'point("' || centroid.ST_X() || ',' || centroid.ST_Y() || '")',
    description: random_string(250)
ORDER BY hash(rowid + 43)

Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [18]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb'

id,location,description
117886,"point(""-85.6466748899667,39.28365177477235"")",uRLfaxtVNZhy5wa1C5KXXJ3IAlv0guNMfk1j0Kq76H2ARJd0FLfZr4H7AxjP25BlypqQI2vWeH3eqmuEbZQopM6J1MPorCei8EINftETubyzIrMCHqiFUlM0hEwNevQYS9G6ivpuhNgkbyTYvXQl8DmHbxqKZlRVwC5xWrY0a6DZmQn0CH690d7rQPtPxR2QLKeIudLa49O6aFtXVyLLTEh9zkOn0qDmkHQ4RLZa0JSxJSdYUvqLTuWK9k
123815,"point(""139.0385562473188,36.42874679055316"")",f7zwXwx3YOCefwNsJy1o40aRznbyzWd2deJ9cVSE7YhdY6s3MRot0b3deDMs3DPjn164p742cxYNn1yoedNcKg7irrRHPLOO8nefXcXlzHbGaE5DBxVLOe6onxDHICUarbPOHn7T0vjl0oXggqTGmZeRdEEr2cGaBKSHmtBRhDNfWUveLKYLPaKPlZEpJ49L74zPb1NANGsfn7N2W6IPvcBu4W2m3W5F9LpZlzzERFkrHXFLBuhI5ORGu5
24588,"point(""99.10831835077646,11.923422147551403"")",qcNNOg0w9qSmL4PwQZb9hKkfIwaxwaxVFwSOjzjQhCwYED2m1q4RRmrZBtMgxQcUt5KEMiECtnhaFCXMhMOlxDlTf4Ve0a9GD9v6w04q0a5AeNUbPRp2kXAqqSMmlkFNlQ31P7EyP5scX6nADt44HN5KlGVojE7PLBxSDJCVtrmtp3mWZH0DnFB6srdS2GHJJ1R0vmvvJ2whRnZMb5hEUwpwrd7kyPse1RCSO8ntGcpgjzOBcK12oMEeGu
398352,"point(""37.69955308014503,54.3793027489593"")",YYDDWmDnVKlgyVWm8YAxFXknFl8om4EOcF7L9r7xL7aqQy4M3YCZeQz3VWHa2FtD12cibUEI5HpFNgtdRjwZdQ8fs7f2f4DvLTO4rl7ZNtwE6cTXdM39QrNx9a9NBZWmOjCWpoVDdQ2QoB4aTTATw2ZFuQjglsirYaNeugScwrphPlGLVnZPacnDnRE2JSaa0WvhCz1ClhGr5CJoseMu2F43P6GjNb6rsBDX3KmChP46XREH15JB5T1T0P
80284,"point(""-8.084735554647613,5.9921804183259475"")",PIfl2tL5OEAIqH4Ap2MzT1rxTkT9yPYt5kmCiYmvCnQHYCsTU7Er7iFn1fszzFShKEcv3KMYZ1qvrDpOzcRfc5Mc2mOQOSNp5veS6wFz6JP32rcsy2JXfGC92jc025CWoRtdhCSS8VnTI8Pb4C1hqdyZiWEdSE9lSzpkJY5bbvaJWlasKEXmNxqmJtMqfkA8sW8WidmJjzwemnj89doVnTmOSFeMVMDvBmpRwLiFQF37pEfmCi4Hppg1bg
113274,"point(""139.44185915948967,35.90150919549181"")",jnTG2Bvn9cgmg6UDqQW6DpFn0DmpT6lssqQP3VAK3Fickbguxo1NgoEItCZzYizYaLO4A8SO2JFEPwmvgdnDLRcc0z41Rvso12KmczYoCmWiHvGFqsRCZLKdFqnM7qsbtEzBtAyyP61Jq0ccaVboU1xS9RoEciU7VNrKQBfU3gpkBCnixqkoHRY1wdKO7xjZYszrAasoY2PwfxTEKP3reOXRZVjZOLaFcge7IMoyrJYreVzxeaHSmWCLE4
91013,"point(""36.06921455300512,50.51046800739962"")",DABe2K9IQOfzAz3lzY1aEoZtLyXzvdh5qv8icrGldSf9gHBwxUoiaAaRVJkLJnLv6iW2y3Rjmex3DwgjiRAhgd9M8E7qH0XD4gITIhukQUbXqOhYk1Smmp95Ex17XVFOiViNw5TjU0tSwNdDTcxuTeuHAY7kWN3We6yrVKr2EHaabJ7VmflOdItxuFw1t55qSwVikJCKOzzvmCChSWIJdqmjifUrSt7OoStqp6fEWnhYZWk2QS37z0tljX
238267,"point(""-117.28437550489467,34.114246342027826"")",OfTcO3pVaKFeBjy8h1hIEA5iaeQvd4zgRh7c2Wwo3rqC2FtMvwgfFIRq86zS2KpbDF25rihm4DKdiL5SOOQTBUY35BnVSg4c0q5UELYfUWB2W5dmA3IjKrhBcBxEcPmG3bj3mw3XVM3uZZc4Kea1k57vT0FAoerbyjDcCPRe1FmwN75kaq4geQfLpWmup0jDayfuK5niuw0Bo9wo3puV9cZEdSGfHxEypZvSfol8s9nnEM1qUUQ0KSct1y
130264,"point(""-83.69704215677237,41.71782106521608"")",lyfEHeBG5vQUjQe8Vv7ZlezGU7SIiHnfQ1y6N8lyWrbL0R8eNoFQPn9FqPYKAqQhxUxeMhahK7WofTEdYZWd5CkHuanqzXIe97gUV1nypcxnbrwBo3YLMfUlgXKOVV5tHJGkvh4cvNyEk7oopo5f34J4idw9dUfmvspNOIuaQBKYpZ5BogdQb8BN7wbyuHalOo8jy7YsSY8gR6yoOAv04RbAA9jwaFT2m8CCmHJ7oaoIerGI73h4X41HVi
13305,"point(""-70.88601036685225,18.564127708683035"")",I6qwsDmxagtQcVKULuxvS9fqncYSi8je42gEzldrEojvpOWsh391TtBLKD3PSmEaG4cjRfnj6GauQ6qa06lka4TSCncO6xzpmY86ORDJ2ghjiBgi0N9zD2b0RxJ0jmmQlgpmjlZEs4PVc3mRqBXv2gFK7IoeyCX1Yvsi7rfldrl1hcvWpcBX7DVUgACmGEkF6PTgrxAPMhvIPKGh9H88jtnvpoqNP2zcEIPu2nNLRChMQVqSPz3O5Q8TI2


In [19]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb'

count_star()
402325


In [20]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,402325,377825,201163.0,116141.36786333578,100825,201153,301840,402325,0.00
location,VARCHAR,"point(""-0.00022686332641463965,44.553858467915205"")","point(""99.99367990063318,9.730698042662416"")",327363,None,None,None,None,None,402325,0.00
description,VARCHAR,0006Lph3A7Vu5c61AVwif7tinSPLGwKSha4dcZ9pT9gA2uxxd1Otuiia4r5Nfw4xMlC4SxEZIkU03MSWFiRNoSjbSiAQbMq0GzSAtrniUS1WObfqmJUuqXdu7G0aCc3cQfpiCGh28935vntmuAfV0lltjMlaviVPp9UH2RGMJGWnk4RnVxM1nA7j0l39Paoz5i2PrmGlWLtqc9clXKQrUcDcBPOcswJMM2gS994pifqesbmz33niFaKvEa,zzxiklsHLTZYEZfHkGDhm4hhJQQNh8o44aKXhK6W6zWmriVELVya6KAFvFyDDmOVoh9dBaaV8euSfzGkFTfJtRFpZPa0kMEuG35vzNRx561BjroZUKZ20z9gftdCyga3GML20IKRsxwgDzkDzwMgBzIMO12qRUwSIXchhfJG7DwB4lsJupiIHn6CSVOIDV4bCYvd1tZMjaOmw76QuUNavsryBbOoLhWzNcTmF6nNfcXieSpemwXa7L6pob,420489,None,None,None,None,None,402325,0.00


In [24]:
export_filename = input_filename.replace(".geojson", ".json")
export_path = "./data/json/" + export_filename

os.makedirs(os.path.dirname(export_path), exist_ok=True)

print(f"Exporting to {export_filename}")
print(f"Exporting to {export_path}")

%sql COPY (SELECT * FROM point_data_centroid) TO '{{export_path}}' (FORMAT JSON);

Exporting to cemetery.json
Exporting to ./data/json/cemetery.json


Running query in 'duckdb'

Count


In [25]:
# replace "point(\"<valueX>,<valueY>\")" with point("139.6499560172321,35.57251104983658")
new_export_path = export_path.replace("json", "adm")
with open(new_export_path, "w") as f:
    for row in open(export_path, "r").readlines():
        new_row = row.replace('location":"point(\\"', 'location":point("').replace('\\")","description"', '"),"description"')
        f.write(new_row)